# Primary Data Wrangling Notebook for Data Visualization Final Project

In [1]:
# Import pertinent libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import os

# Provide plotting in-line as needed
%matplotlib inline

In [2]:
# create data path for importing data
path = "./data/"

# Join all files in the data path into a list
all_files = glob.glob(os.path.join(path, "*.csv"))

# Create a for loop to extract just the filename (not extension)
# for use in naming dataframes
name_list = []
for file in all_files:
    name_list.append(os.path.splitext(os.path.basename(file))[0])


In [3]:
# create an empty dictionary to hold all the dataframes
d = {}

# create a for loop to recursively add .csv files to the dictionary as dataframes
for idx, val in enumerate(all_files):
    d[name_list[idx]] = pd.read_csv(val)

In [4]:
# create a series of dictionaries from the 'Categories' dataframe
# These dictionaries will be used to create a larger dataframe
# The reason for this is that the data came from a relational database
# that really didn't need to be relational. Having it as a flat datafram
# makes EDA and data visualization much easier
cats = d['categories'].set_index('id')['cat_name'].to_dict()
cats[-1] = 'Other'
styles = d['styles'].set_index('id')['style_name'].to_dict()
styles[-1] = 'Other'
breweries = d['breweries'].set_index('id')['name'].to_dict()

# Use the dictionaries above to replace the foreign keys in 
# the 'beers' dataframe with their actual values
d['beers'].cat_id.replace(cats, inplace = True)
d['beers'].style_id.replace(styles, inplace = True)
brewery_name_list = list(d['beers'].brewery_id.replace(breweries))

# Use brewery_name_list to create a new column called brewery_name
d['beers']['brewery_name'] = brewery_name_list

# Check our work
d['beers'].head()

,id,brewery_id,name,cat_id,style_id,abv,brewery_name
0,1,812,Hocus Pocus,Other Style,Light American Wheat Ale or Lager,4.5,Magic Hat
1,2,264,Grimbergen Blonde,Other,Other,6.7,Brouwerij Alken-Maes
2,3,779,Widdershins Barleywine,Other,Other,9.1,Left Hand Brewing Company
3,4,287,Lucifer,Other,Other,8.5,Brouwerij Liefmans
4,5,1056,Bitter,Other,Other,4.0,Ridgeway Brewing


In [5]:
# Merge the 'beers' data frame with the 'breweries_geocode' dataframe.
# In this case, the 'brewery_id' field served as the key for joining
df_beers = d['beers'].merge(d['breweries_geocode'], on = 'brewery_id')

# Drop redundant and uncessary columns
df_beers.drop(columns=['id_x', 'id_y', 'accuracy'], inplace = True)

# Sort values by brewery_id for a cleaner presentation
df_beers.sort_values('brewery_id', inplace = True)

# Check our work
df_beers

,brewery_id,name,cat_id,style_id,abv,brewery_name,latitude,longitude
5652,1,(512) Pale,North American Ale,American-Style Pale Ale,5.8,(512) Brewing Company,30.2234,-97.7697
5656,1,(512) Bruin,North American Ale,American-Style Brown Ale,7.6,(512) Brewing Company,30.2234,-97.7697
5657,1,(512) Whiskey Barrel Aged Double Pecan Porter,Irish Ale,Porter,8.2,(512) Brewing Company,30.2234,-97.7697
5658,1,One,Belgian and French Ale,Belgian-Style Pale Strong Ale,8.0,(512) Brewing Company,30.2234,-97.7697
5653,1,(512) IPA,North American Ale,American-Style India Pale Ale,7.0,(512) Brewing Company,30.2234,-97.7697
5655,1,(512) ALT,German Ale,German-Style Brown Ale/Altbier,6.0,(512) Brewing Company,30.2234,-97.7697
5654,1,(512) Pecan Porter,Irish Ale,Porter,6.8,(512) Brewing Company,30.2234,-97.7697
5651,1,(512) Wit,Belgian and French Ale,Belgian-Style White,5.2,(512) Brewing Company,30.2234,-97.7697
4727,2,Amendment Pale Ale,North American Ale,American-Style Pale Ale,5.2,21st Amendment Brewery Cafe,37.7825,-122.3930
4729,2,North Star Red,North American Ale,American-Style Amber/Red Ale,5.8,21st Amendment Brewery Cafe,37.7825,-122.3930


In [9]:
# Output the new 'non-relational' table to a csv file for future EDA/Visualization
df_beers.to_csv('./data/df_beers.csv', index = False)

# End of Notebook 1
### As it turns out, I didn't do much wrangling here - this was primarily cleaning one set of tables to create a master table. More wrangling is done in the Data_Viz notebook. Stay Tuned!